# Curso de Optimización I
## CIMAT; DEMAT, UG

## Parcial 2
## Angel Antonio Méndez Hernández

| Descripción:                         | Fechas               |
|--------------------------------------|----------------------|
| Fecha de publicación del documento:  | **Mayo 16, 2024**    |
| Hora de inicio:                      | **11:00**            |
| Hora límite de entrega:              | **12:30**            |


### Indicaciones

Lea con cuidado los ejercicios.

Puede usar las notas de clase y las tareas hechas para resolver el examen.

Al final, entregue el notebook con sus respuestas, junto con los códigos que hagan falta para reproducir los resultados. Si es más de un archivo, genere un archivo ZIP que contenga el notebook y los scripts adicionales.

A partir del notebook genere un archivo PDF con las respuestas y envíelo por separado 
antes de la hora límite.

---


## Ejercicio 1. (6 puntos)

Considere el problema 

$$ \min\; f(\mathbf{x}) \quad \text{sujeto a} \quad c_1(\mathbf{x})=0.$$

Encontrar la solución usando un penalización cuadrática (clase 29). 
Para esto contruimos la función

$$
Q(\mathbf{x}; \mu) =  f(\mathbf{x}) + \frac{\mu}{2 }(c_1(\mathbf{x}) )^2
$$

1. Programar la función $Q(x; \mu)$ y su gradiente 

$$\nabla Q(\mathbf{x}; \mu) = \nabla f(\mathbf{x}) + \mu c_1(\mathbf{x}) \nabla c_1(\mathbf{x}).$$

2. Programar el método de penalización cuadrática usando el método BFGS modificado:

a) Dar la función $f(\mathbf{x})$, $c_1(\mathbf{x})$, la función $Q(\mathbf{x}; \mu)$, su gradiente $\nabla Q(\mathbf{x}; \mu)$, 
   un punto inicial $\mathbf{x}_0$,  $\mu_0$, una
   tolerancia $\tau>0$, el número máximo de iteraciones $N$, y los parámetros que se necesiten para usar el método BFGS modificado.

b) Para $k=0, 1, ..., N$ repetir los siguientes pasos:

b1) Definir $\tau_k = \left(1 + \frac{10N}{10k+1} \right)\tau$

b2) Calcular el punto  $\mathbf{x}_{k+1}$ como el minimizador de $Q(\mathbf{x}; \mu_k)$ 
    con el método BFGS modificado usando como punto inicial a $\mathbf{x}_{k}$ y la tolerancia $\tau_k$.

b3) Imprimir el punto $\mathbf{x}_{k+1}$, $f(\mathbf{x}_{k+1})$, $Q(\mathbf{x}; \mu_k)$,
    el número de iteraciones realizó el algoritmo BFGS
    y el valor  $c_1(\mathbf{x}_{k+1})$.

b4) Si $\|\mathbf{x}_{k+1} - \mathbf{x}_k\|<\tau$, terminar devolviendo  $\mathbf{x}_{k+1}$ 

b5) En caso contrario, hacer $\mu_{k+1} = 2\mu_k$ y volver al paso (b1)

3. Probar el algoritmo tomando como $f$ a la función de Beale, $c_1(\mathbf{x}) = x_1^2 + x_2^2 - 4$, 
   $\mu_0=0.5$, $N=1000$ y $\tau=\epsilon_m^{1/3}$.
   
   Use los puntos iniciales $\mathbf{x}_0 = (0, 2)$ y $\mathbf{x}_0 = (0, -2)$.

4. Para verificar el resultado obtenido  haga lo siguiente:

- Genere una partición $\theta_0 < \theta_1 < ... \theta_m$ del intervalo $[0, 2\pi]$ con $m=1000$
- Evalue la función de Beale en los puntos $(2\cos \theta_i, 2\sin \theta_i)$ para $i=0, 1, ..., m$.  
  e imprima el punto en donde la función tuvo el menor valor y el valor de la función
  en ese punto.
   
   
**Nota:** Si no tiene implementado el método BFGS modificado, puede elegir otro método de optimización,
pero se aplica una penalización de 0.5 puntos.

### Solución:

In [1]:
# Librerías
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Épsilon de la máquina
eps = np.finfo(float).eps
eps

2.220446049250313e-16

In [3]:
def beale(x):
  return (1.5-x[0]+x[1]*x[0])**2 + (2.25-x[0]+x[0]*x[1]**2)**2 + (2.625-x[0]+x[0]*x[1]**3)**2

def Grad_beale(x):
  d1=2*(x[1]-1)*(1.5-x[0]+x[1]*x[0])+2*(x[1]**2-1)*(2.25-x[0]+x[0]*x[1]**2)+2*(x[1]**3-1)*(2.625-x[0]+x[0]*x[1]**3)
  d2=2*(x[0])*(1.5-x[0]+x[1]*x[0])+4*(x[0]*x[1])*(2.25-x[0]+x[0]*x[1]**2)+6*(x[0]*x[1]**2)*(2.625-x[0]+x[0]*x[1]**3)
  return np.array([d1, d2])

def Hes_beale(x):
  d1=2*(x[1]**6 +x[1]**4 -2*x[1]**3 -x[1]**2 -2*x[1] +3)
  d2=4*x[0]*(3*(x[1]**5) +2*(x[1]**3) -3*(x[1]**2) -x[1] -1) +15.75*(x[1]**2) +9*x[1] +3
  d3=6*x[0]*(5*x[0]*(x[1]**4 +0.4*(x[1]**2) -0.4*x[1] -0.066666) +5.25*x[1] +1.5)
  return np.array([[d1, d2], [d2, d3]])

In [6]:
def backtracking(a0, r, c, xk, f, df, pk, maxIter):
  a=a0
  for i in range(maxIter):
    if f(xk+a*pk)<= f(xk) + c*a*(np.dot(df(xk), pk)):
      return a, i+1, True
    a=r*a
  return a, maxIter, False

In [15]:
# Método BFGS
# args = r, c1, maxIter2
def bfgs(f, grad, x0, tol, H0, maxIter1, *args, op=0):
    xk = x0
    dim = len(xk)
    Hk = H0
    gk = grad(xk)
    identidad = np.identity(dim)
    alpha = 1
    for k in range(maxIter1):
        aux = np.dot(gk, gk)
        if np.sqrt(aux) < tol:
            return xk, dim, f(xk), np.sqrt(aux), k+1, True
        pk = -Hk@gk
        aux1 = np.dot(pk, gk)
        if aux1 > 0:
            l1 = 10e-5 + aux1/aux
            Hk = Hk + l1*identidad
            pk = pk - l1*gk
        aux2 = backtracking(alpha, args[0], args[1], xk, f, grad, pk, args[2])
        #if aux2[-1] == False:
            #print("Backtracking está consumiendo todas las iteraciones.")
        alpha = aux2[0]
        xk1 = xk + alpha*pk
        gk1 = grad(xk1)
        sk = xk1 - xk
        yk = gk1 - gk
        aux4 = np.dot(yk,yk)
        if op==1:
          if np.sqrt(aux4) < tol:
              print("Termina por minima diferencia en gradientes.")
              return xk, dim, f(xk), np.sqrt(aux), k+1, True
        aux3 = np.dot(yk, sk)
        if aux3 <= 0:
            l2 = 10e-5 - aux3/aux4
            Hk = Hk + l2*identidad
        else:
            rho = 1/aux3
            Hk = (identidad -rho*(sk@yk.T))@Hk@(identidad -rho*(yk@sk.T)) +rho*(sk@sk.T)
        gk = gk1
        xk = xk1
    return xk, dim, f(xk), np.sqrt(aux), maxIter1, False

# Función para resultados
def results_bfgs(sol):
    print(f"Dimension: {sol[1]}")
    print(f"¿Terminó por criterio de paro?: {sol[-1]}")
    print(f"Número de iteraciones realizadas: {sol[-2]}")
    print("La solución alcanzada es: ",sol[0][:4],"...", sol[0][-4:])
    print(f"Q(xk)={sol[2]}")
    print(f"||Grad_f(xk)||={sol[3]}")

In [5]:
def c1(x):
    return x[0]**2 + x[1]**2 -4
def grad_c1(x):
    g1 = 2*x[0]
    g2 = 2*x[1]
    return np.array([g1, g2])

In [7]:
def Q(x, mu, f, c):
    return f(x) + (mu*(c(x))**2)/2.0

def grad_Q(x, mu, gradf, gradc, c):
    return gradf(x) + mu*c(x)*gradc(x)

In [22]:
def penalización(f, c1, Q, gradQ, x0, mu0, tol, maxIter):
    xk = x0.copy()
    bres = 0
    muk=mu0
    for k in range(maxIter):
        tk = (1+ (10*maxIter)/(10*k +1))*tol
        def gQ(x):
            return gradQ(x, muk, Grad_beale, grad_c1, c1)
        sol = bfgs(lambda x: Q(x, muk, beale, c1), gQ, xk, tk, np.identity(len(xk)), 5000, 0.5, 0.001, 500)
        #results_bfgs(sol)
        xk1 = sol[0]
        print("f(xk)=", f(xk1))
        print("c1(xk)=", c1(xk1))
        print("\n")
        if np.linalg.norm(xk1 - xk) < tol:
            bres = 1
            return xk1
        xk = xk1
        muk = 2*muk
    return xk

In [20]:
penalización(beale, c1, Q, grad_Q, np.array([0,2]), 0.5, eps**(1/3.0), 1000)

Dimension: 2
¿Terminó por criterio de paro?: False
Número de iteraciones realizadas: 5000
La solución alcanzada es:  [-1.47379206  1.44983047] ... [-1.47379206  1.44983047]
Q(xk)=1.265323123369399
||Grad_f(xk)||=0.1703721279825062
f(xk)= 1.246544337406337
c1(xk)= 0.27407142107897364


Dimension: 2
¿Terminó por criterio de paro?: True
Número de iteraciones realizadas: 90
La solución alcanzada es:  [-1.43870845  1.45749005] ... [-1.43870845  1.45749005]
Q(xk)=1.2801749811237002
||Grad_f(xk)||=0.005068707771168922
f(xk)= 1.2613260739287522
c1(xk)= 0.1941592500755398


Dimension: 2
¿Terminó por criterio de paro?: True
Número de iteraciones realizadas: 66
La solución alcanzada es:  [-1.39843106  1.46659966] ... [-1.39843106  1.46659966]
Q(xk)=1.2903785403480708
||Grad_f(xk)||=0.002856471160577382
f(xk)= 1.279031177331501
c1(xk)= 0.10652400206793633


Dimension: 2
¿Terminó por criterio de paro?: True
Número de iteraciones realizadas: 45
La solución alcanzada es:  [-1.3744578   1.47218682] ..

array([-1.34645062,  1.47887732])

In [21]:
penalización(beale, c1, Q, grad_Q, np.array([0,-2]), 0.5, eps**(1/3.0), 1000)

Dimension: 2
¿Terminó por criterio de paro?: True
Número de iteraciones realizadas: 57
La solución alcanzada es:  [2.12445767 0.20473748] ... [2.12445767 0.20473748]
Q(xk)=0.4281608102332176
||Grad_f(xk)||=0.05805262884026415
f(xk)= 0.35108854833885317
c1(xk)= 0.5552378297427669


Dimension: 2
¿Terminó por criterio de paro?: True
Número de iteraciones realizadas: 13
La solución alcanzada es:  [2.06980403 0.18697798] ... [2.06980403 0.18697798]
Q(xk)=0.47154558103715394
||Grad_f(xk)||=0.0007064648101836795
f(xk)= 0.42064929152090713
c1(xk)= 0.31904949307669117


Dimension: 2
¿Terminó por criterio de paro?: True
Número de iteraciones realizadas: 8
La solución alcanzada es:  [2.03594249 0.17444807] ... [2.03594249 0.17444807]
Q(xk)=0.4995019656180876
||Grad_f(xk)||=0.0017514851358031045
f(xk)= 0.4687038418708266
c1(xk)= 0.17549394219533898


Dimension: 2
¿Terminó por criterio de paro?: True
Número de iteraciones realizadas: 14
La solución alcanzada es:  [2.0161352  0.16712924] ... [2.0161

array([1.99368422, 0.15885616])

In [27]:
theta =np.linspace(0, 2*np.pi, 1000)
puntos = [[2*np.cos(t), 2*np.sin(t)] for t in theta]

puntos_eval = [beale(p) for p in puntos]

indice_minimo = np.argmin(puntos_eval)
punto_minimo = puntos[indice_minimo]

# Imprimir el valor mínimo y el punto correspondiente
print("El valor mínimo es:", puntos_eval[indice_minimo])
print("El punto correspondiente es:", punto_minimo)

El valor mínimo es: 0.5342196283845664
El punto correspondiente es: [1.9933185071907058, 0.16334420372641326]


_

```







```

---

## Ejercicio 2. (4 puntos)

Programar el método de Newton para resolver el sistema de ecuaciones no lineales
(Algoritmo 1 de la Clase 24):

$$ \begin{array}{rcl}
 2x_0 + x_1 &=& 5 - 2x_2^2 \\
    x_1^3 + 4x_2 &=& 4 \\
    x_0 x_1 + x_2 &=& \exp(x_2)
   \end{array}
$$

1. Programar la función $\mathbf{F}(\mathbf{x})$ correspondiente a este sistema de ecuaciones y 
   su Jacobiana $\mathbf{J}(\mathbf{x})$ 
2. Programe el algoritmo del método de Newton. Use como condición de paro que el ciclo termine
   cuando $\|\mathbf{F}(\mathbf{x}_k)\|< \tau$, para una tolerancia $\tau$ dada.
   Haga que el algoritmo devuelva el punto $\mathbf{x}_k$, el número de iteraciones $k$,
   el valor $\|\mathbf{F}(\mathbf{x}_k)\|$ y una variable indicadora $bres$ que es $1$
   si se cumplió el criterio de paro o $0$ si terminó por iteraciones.
3. Para probar el algoritmo y tratar de encontrar varias raíces, 
   haga un ciclo para hacer 20 iteraciones y en cada iteración haga lo siguiente:
   
- Dé el punto inicial $\mathbf{x}_0$ como un punto aleatorio generado con `numpy.random.randn(3)`
- Ejecute el método de Newton usando $\mathbf{x}_0$, la tolerancia $\tau = \sqrt{\epsilon_m}$ y 
  un máximo de iteraciones $N=100$.
- Imprima el punto $\mathbf{x}_k$ que devuelve el algoritmo, la cantidad
  de iteraciones realizadas, el valor de $\|\mathbf{F}(\mathbf{x}_k)\|$ y la variable
  indicadora $bres$.


### Solución:

In [28]:
def F(x):
    f1 = 2*x[0] + x[1] - 5 +2*x[2]**2
    f2 = x[1]**3 + 4*x[2] - 4
    f3 = x[0]*x[1] + x[2] - np.exp(x[2])
    return np.array([f1, f2, f3])

def JF(x):
    df1_dx0 = 2
    df1_dx1 = 1
    df1_dx2 = 4*x[2]
    
    df2_dx0 = 0
    df2_dx1 = 3*x[1]**2
    df2_dx2 = 4

    df3_dx0 = x[1]
    df3_dx1 = x[0]
    df3_dx2 = 1 - np.exp(x[2])

    jacobiana = np.array([
        [df1_dx0, df1_dx1, df1_dx2],
        [df2_dx0, df2_dx1, df2_dx2],
        [df3_dx0, df3_dx1, df3_dx2]
    ])

    return jacobiana

In [29]:
def newton(f, jf, x0, tol, maxIter):
    xk = x0.copy()
    for k in range(maxIter):
        fk = f(xk)
        fkn = np.linalg.norm(fk)
        if fkn < tol:
            return xk, k+1, fkn, 1
        jk = jf(xk)
        sk = np.linalg.solve(jk, -fk)
        xk = xk + sk
    return xk, maxIter, fkn, 0

In [31]:
for i in range(20):
    x0 = np.random.randn(3)
    sol = newton(F, JF, x0, np.sqrt(eps), 100)
    print(f"¿Terminó por criterio de paro?: {sol[-1]}")
    print(f"Número de iteraciones realizadas: {sol[1]}")
    print("La solución alcanzada es: ",sol[0])
    print("||fk||=", sol[2])
    print("\n")

¿Terminó por criterio de paro?: 1
Número de iteraciones realizadas: 6
La solución alcanzada es:  [1.42246939 0.97538853 0.76800804]
||fk||= 8.759973326466135e-10


¿Terminó por criterio de paro?: 1
Número de iteraciones realizadas: 8
La solución alcanzada es:  [ 0.66819062  1.97278644 -0.91946515]
||fk||= 2.7544560071964744e-11


¿Terminó por criterio de paro?: 1
Número de iteraciones realizadas: 6
La solución alcanzada es:  [1.42246939 0.97538853 0.76800804]
||fk||= 3.3604419030518293e-09


¿Terminó por criterio de paro?: 1
Número de iteraciones realizadas: 8
La solución alcanzada es:  [1.42246939 0.97538853 0.76800804]
||fk||= 3.0517597567390576e-12


¿Terminó por criterio de paro?: 1
Número de iteraciones realizadas: 9
La solución alcanzada es:  [1.42246939 0.97538853 0.76800804]
||fk||= 1.5058867625804301e-09


¿Terminó por criterio de paro?: 1
Número de iteraciones realizadas: 6
La solución alcanzada es:  [1.42246939 0.97538853 0.76800804]
||fk||= 2.8694106147801977e-10


¿Terminó

_

```







```

---